# Primera entrega
Luis Pedro Montenegro - 21699


## Limpieza de datos
En esta entrega se hará una recolección y limpieza de datos solamente. No habrá modelado ni análisis. 
<br>Solamente velar por la integridad, coherencia y cohesión del conjunto de datos.facilidad de manejo.

In [ ]:
# !pip install -r requirements.txt

In [1]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm

### Conversión de .xls a .csv

Debido a que los archivos crudos descargados desde la página del Mineduc vienen en formato .xls formateado como .html tenemos que hacer cambio de eso.<br>
Esto por múltiples razones:
- Mejorar la estructura de los datos.
- Preservar solamente la información requerida (los .xls almaceban más páginas e información irrelevante)
- Mayor falicidad de usar y obtener los datos si están en formato delimitado por comas.
- Tener datos más ligeros ya que no acarreamos con mucha información innecesaria. 

In [11]:
def transform_html_to_csv(input_path: str, output_path: str=None, tables_to_convert: list[int] = [9]) -> None:
    '''
    Turns .html files into .csv files

    Params:
        input_path: where the .html files are stored
        output_path: where the .csv will be stored
    Returns:
        out: None
    '''
    if output_path is None:
        output_path = input_path 
    else:
        os.makedirs(output_path, exist_ok=True)

    for filename in tqdm(os.listdir(input_path), desc='Converting Files'):
        if filename.endswith('.xls'):
            html_path:str = os.path.join(input_path, filename)
            base_name:str = os.path.splitext(filename)[0]
            # try to read the excel and copy it into a .csv file
            try: 
                tables:list[pd.DataFrame] = pd.read_html(html_path)
                # if no tables, skip
                if not tables:
                    print(f"No tables found in '{filename}'")
                    continue

                # convert each of the tables selected to .csv
                for i, df in enumerate(tables):
                    if i in tables_to_convert:
                        csv_filename = f"{base_name}_table{i}.csv"
                        csv_path = os.path.join(output_path, csv_filename)
                        df.to_csv(csv_path, index=False)
                        print(f"Converted '{filename}' to '{csv_filename}'") # print all of the transformations of the .xls to its respective .csv files
                    
            except Exception as e:
                print(f"Error processing file '{filename}' : {e}")

def remove_files(path: str, extension: str) -> None:
    '''
    Removes all files that have a certain extension
    Input:
        path: folder path where .xls to be removed are stored
    '''
    file_list: list[str] = os.listdir(path)
    if not file_list:
        print(f"No files removed since there was none found: '{path}'")
        return

    for filename in file_list:
    
        if filename.endswith(extension):
            try:
                xls_path = os.path.join(path, filename)
                os.remove(xls_path)
            except Exception as e:
                print(f"Error removing file '{filename}' : {e}")


In [5]:
# start by turning .xls files to .csv files
transform_html_to_csv(input_path="../Dataset_raw", output_path="../Dataset_cleaned", tables_to_convert=[9])

Converting Files:   2%|▏         | 1/41 [00:00<00:14,  2.83it/s]

Converted 'alta_verapaz_preprimaria.xls' to 'alta_verapaz_preprimaria_table9.csv'


Converting Files:   7%|▋         | 3/41 [00:00<00:11,  3.36it/s]

Converted 'alta_verapaz_primaria.xls' to 'alta_verapaz_primaria_table9.csv'
Converted 'baja_verapaz_preprimaria.xls' to 'baja_verapaz_preprimaria_table9.csv'


Converting Files:  12%|█▏        | 5/41 [00:01<00:07,  4.63it/s]

...

Converted 'solola_primaria.xls' to 'solola_primaria_table9.csv'
Converted 'suchitepequez_preprimaria.xls' to 'suchitepequez_preprimaria_table9.csv'
Converted 'suchitepequez_primaria.xls' to 'suchitepequez_primaria_table9.csv'


Converting Files:  98%|█████████▊| 40/41 [00:06<00:00,  8.53it/s]

Converted 'totonicapan_preprimaria.xls' to 'totonicapan_preprimaria_table9.csv'
Converted 'totonicapan_primaria.xls' to 'totonicapan_primaria_table9.csv'


Converting Files: 100%|██████████| 41/41 [00:06<00:00,  6.63it/s]

Converted 'zacapa_primaria.xls' to 'zacapa_primaria_table9.csv'


In [10]:
# remove .xls files if necessary
remove_files(path="../Dataset_raw", extension='.xls')

No files removed since there was none found: '../Dataset_raw'


### Plan para limpieza de datos
Posteriormente a la transformación de formato y filtración a solo datos crudos de nuestro interés, procederemos a limpiar los datos en sí. <br>
1. Verificaremos si existe datos NA o Null en los archivos. Llenaremos los datos faltantes con datos artificiales ya que no podemos eliminar ningún registro <br>
debido a que si eliminamos alguno, sería una sede faltante. Cosa que afectaría de gran manera el conjunto de datos.
2. Homogenizaremos la información. Es decir, que todos los archivos tengan el mismo formato i.e. todos los nombres estén escritos igual, los apellidos que tienen tildes se escriban igual en cada archivo,
que las palabras vengan o solo en mayúsculas, solo en minúsculas, etc.
3. Identificaremos las columnas que más trabajo de reajuste necesiten.
- Nombres (Supervisor, Director, Establecimiento, Sector): debido a que puede que exista apellidos iguales pero escritos distinto y eso genere problemas a la hora de verlos. Digamos, puede que Hernández aparezca con tilde en unos registros y en otros no. También es bueno verificar que estén escritos todos o en mayúsculas o en minúsculas (o todos iguales), ya que digamos, si tenemos Santa rosa en un registro y Santa Rosa en otro, a la hora de hacer un encoding estos resultarán con dos valores distintos. 
- Dirección: Asegurarnos que las direcciones lleven una estructura similar.
- Códigos: Verificar que los códigos de los registros sean únicos y evitar tener repetidos.

